# Housing rent prices and venues data analysis of Frankfurt

#### Import all the packages

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip -q install folium
import folium # map rendering library

#!pip -q install plotly

# to retrieve the districts coordinates
import urllib.parse

print('Libraries imported.')

Libraries imported.


## Data preparation

### Retrieve list of Frankfurt's districts from Wikipedia's page

In [8]:
# Webpage url (old link as suggested in the Discussion Forums)                                                                                                             
url = 'https://de.wikipedia.org/wiki/Liste_der_Stadtteile_von_Frankfurt_am_Main'

# Extract tables using pandas
dfs = pd.read_html(url,header=[0],flavor='bs4')

# Get first table                                                                                                           
df = dfs[0]

df

,Nr.,Stadtteil,Fläche[3]in km²,Einwohner,Weiblich,Männlich,Deutsche,Ausländer,Ausländerin Prozent,Einwohnerje km²,Ortsbezirk,Stadtgebietseit,Vorherige Zugehörigkeit
0,1,Altstadt,506,4.218,2.065,2.153,2.669,1.549,367,8336,01 Innenstadt I,1866[Anm. 1],Freie Stadt Frankfurt
1,2,Innenstadt,1491,6.599,3.112,3.487,3.539,3.060,464,4426,01 Innenstadt I,1866[Anm. 2],Freie Stadt Frankfurt
2,3,Bahnhofsviertel,542,3.552,1.321,2.231,1.706,1.846,52,6554,01 Innenstadt I,1866[Anm. 3],Freie Stadt Frankfurt
3,4,Westend-Süd,2497,19.314,9.839,9.475,14.006,5.308,275,7735,02 Innenstadt II,1866[Anm. 3],Freie Stadt Frankfurt
4,5,Westend-Nord,1632,10.373,5.391,4.982,7.366,3.007,29,6356,02 Innenstadt II,1866[Anm. 3],Freie Stadt Frankfurt
5,6,Nordend-West,3100,30.897,15.845,15.052,24.144,6.753,219,9967,03 Innenstadt III,1866[Anm. 3],Freie Stadt Frankfurt
6,7,Nordend-Ost,1532,23.182,12.045,11.137,18.016,5.166,223,15132,03 Innenstadt III,1866[Anm. 3],Freie Stadt Frankfurt
7,8,Ostend,5564,29.477,15.108,14.369,21.070,8.407,285,5298,04 Bornheim/Ostend,1866[Anm. 3],Freie Stadt Frankfurt
8,9,Bornheim,2786,30.917,16.316,14.601,23.511,7.406,24,11097,04 Bornheim/Ostend,1877,Stadtkreis Frankfurt am Main[Anm. 4]
9,10,Gutleutviertel,1792,6.964,3.047,3.917,4.000,2.964,426,3886,01 Innenstadt I,1866[Anm. 3],Freie Stadt Frankfurt


In [9]:
df_districts = df.iloc[0:-1,1:8].reset_index(drop=True) # Remove the last row which is the total
df_districts.columns = ['District', 'Surface_km2',"population", "female_pop", "male_pop", "german_pop", "foreign_pop"]

#Fix Flughafen value
df_districts.loc[df_districts.District == "Flughafen", ["population","female_pop", "male_pop", "german_pop", "foreign_pop"]] = df_districts.loc[df_districts.District == "Sachsenhausen-Süd", ["population","female_pop", "male_pop", "german_pop", "foreign_pop"]].values.tolist()

df_districts[["population","female_pop","male_pop","german_pop","foreign_pop"]] = df_districts[["population","female_pop","male_pop","german_pop","foreign_pop"]].apply(lambda x: x.str.replace('.',''))
df_districts = df_districts.astype({'population': 'int64', 'female_pop': 'int64', 'male_pop': 'int64', 'german_pop': 'int64', 'foreign_pop': 'int64'})

df_districts

,District,Surface_km2,population,female_pop,male_pop,german_pop,foreign_pop
0,Altstadt,506,4218,2065,2153,2669,1549
1,Innenstadt,1491,6599,3112,3487,3539,3060
2,Bahnhofsviertel,542,3552,1321,2231,1706,1846
3,Westend-Süd,2497,19314,9839,9475,14006,5308
4,Westend-Nord,1632,10373,5391,4982,7366,3007
5,Nordend-West,3100,30897,15845,15052,24144,6753
6,Nordend-Ost,1532,23182,12045,11137,18016,5166
7,Ostend,5564,29477,15108,14369,21070,8407
8,Bornheim,2786,30917,16316,14601,23511,7406
9,Gutleutviertel,1792,6964,3047,3917,4000,2964


## Get the coordinates of the districts

In [10]:
df_districts["Latitude"] = 0.00000
df_districts["Longitude"] = 0.00000
df_districts["Coordinates"] = np.empty((len(df_districts), 0)).tolist()

for row, district in enumerate(df_districts["District"]):
    downloaded = False
    
    while downloaded == False:
        try:
            print("Checking "+district)
            address = district+', Frankfurt am Main, Germany'
            url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?polygon_geojson=1&format=json'
            #print(url)

            response = requests.get(url).json()
            df_districts["Latitude"][row] = round(float((response[0]["lat"])), 5)
            df_districts["Longitude"][row] = round(float((response[0]["lon"])), 5)
            df_districts["Coordinates"][row] = response[0]["geojson"]['coordinates'][0]
            downloaded = True
        except:
              print("Error with district download - trying again")

#df_districts

import geopandas as gpd 
from shapely.geometry import Polygon
import ast

#df_districts['Coordinates'] = df_districts['Coordinates'].apply(lambda x: ast.literal_eval(x))

df_districts['geometry'] = df_districts['Coordinates'].apply(lambda x: Polygon(x))
df_districts = gpd.GeoDataFrame(df_districts, geometry=df_districts["geometry"])
df_districts.set_crs(epsg=4326, inplace=True)

Checking Altstadt


<ipython-input-10-988ae2dcac80>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_districts["Latitude"][row] = round(float((response[0]["lat"])), 5)
<ipython-input-10-988ae2dcac80>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_districts["Longitude"][row] = round(float((response[0]["lon"])), 5)
<ipython-input-10-988ae2dcac80>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_districts["Coordinates"][row] = response[0]["geoj

Checking Innenstadt
Checking Bahnhofsviertel
Checking Westend-Süd
Checking Westend-Nord
Checking Nordend-West
Checking Nordend-Ost
Checking Ostend
Checking Bornheim
Checking Gutleutviertel
Checking Gallus
Checking Bockenheim
Checking Sachsenhausen-Nord
Checking Sachsenhausen-Süd
Checking Flughafen
Checking Oberrad
Checking Niederrad
Checking Schwanheim
Checking Griesheim
Checking Rödelheim
Checking Hausen
Checking Praunheim
Checking Heddernheim
Checking Niederursel
Checking Ginnheim
Checking Dornbusch
Checking Eschersheim
Checking Eckenheim
Checking Preungesheim
Checking Bonames
Checking Berkersheim
Checking Riederwald
Checking Seckbach
Checking Fechenheim
Checking Höchst
Checking Nied
Checking Sindlingen
Checking Zeilsheim
Checking Unterliederbach
Checking Sossenheim
Checking Nieder-Erlenbach
Checking Kalbach-Riedberg
Checking Harheim
Checking Nieder-Eschbach
Checking Bergen-Enkheim
Checking Frankfurter Berg


,District,Surface_km2,population,female_pop,male_pop,german_pop,foreign_pop,Latitude,Longitude,Coordinates,geometry
0,Altstadt,506,4218,2065,2153,2669,1549,50.11044,8.68235,"[[8.6748666, 50.1092226], [8.6757349, 50.10751...","POLYGON ((8.67487 50.10922, 8.67573 50.10752, ..."
1,Innenstadt,1491,6599,3112,3487,3539,3060,50.11456,8.68359,"[[8.6685139, 50.1132235], [8.6690588, 50.11253...","POLYGON ((8.66851 50.11322, 8.66906 50.11253, ..."
2,Bahnhofsviertel,542,3552,1321,2231,1706,1846,50.10841,8.66815,"[[8.6605492, 50.1095394], [8.6625518, 50.10855...","POLYGON ((8.66055 50.10954, 8.66255 50.10856, ..."
3,Westend-Süd,2497,19314,9839,9475,14006,5308,50.11524,8.66227,"[[8.6433346, 50.1140438], [8.6434858, 50.10917...","POLYGON ((8.64333 50.11404, 8.64349 50.10918, ..."
4,Westend-Nord,1632,10373,5391,4982,7366,3007,50.12636,8.66792,"[[8.6558043, 50.1266272], [8.6564224, 50.12597...","POLYGON ((8.65580 50.12663, 8.65642 50.12597, ..."
5,Nordend-West,3100,30897,15845,15052,24144,6753,50.12491,8.67795,"[[8.6714178, 50.1360727], [8.6719059, 50.13375...","POLYGON ((8.67142 50.13607, 8.67191 50.13375, ..."
6,Nordend-Ost,1532,23182,12045,11137,18016,5166,50.12492,8.69232,"[[8.6887636, 50.1185902], [8.6891034, 50.11829...","POLYGON ((8.68876 50.11859, 8.68910 50.11830, ..."
7,Ostend,5564,29477,15108,14369,21070,8407,50.11237,8.69997,"[[8.6941197, 50.1103561], [8.694791, 50.107693...","POLYGON ((8.69412 50.11036, 8.69479 50.10769, ..."
8,Bornheim,2786,30917,16316,14601,23511,7406,50.12973,8.71061,"[[8.6982605, 50.1391582], [8.7003042, 50.13853...","POLYGON ((8.69826 50.13916, 8.70030 50.13853, ..."
9,Gutleutviertel,1792,6964,3047,3917,4000,2964,50.10171,8.66054,"[[8.6215196, 50.0932461], [8.6219131, 50.08876...","POLYGON ((8.62152 50.09325, 8.62191 50.08877, ..."


## Download data on Rental price index 

In [11]:
# Webpage url (old link as suggested in the Discussion Forums)                                                                                                             
url = 'https://www.miet-check.de/stadtteile_uebersicht.php?stadt=Frankfurt%20am%20Main'

# Extract tables using pandas
dfs = pd.read_html(url,header=[0],flavor='bs4')

# Get first table                                                                                                           
df = dfs[0]

district_rent_price = pd.DataFrame(columns =["District", "Rent_per_sqm"])

district_rent_price = district_rent_price.append(df[["Stadtteile in","Stadtteile in.1"]].rename(columns={"Stadtteile in": "District", "Stadtteile in.1": "Rent_per_sqm"}), ignore_index=True)
district_rent_price = district_rent_price.append(df[["Stadtteile in.2","Stadtteile in.3"]].rename(columns={"Stadtteile in.2": "District", "Stadtteile in.3": "Rent_per_sqm"}), ignore_index=True)
district_rent_price = district_rent_price.append(df[["Stadtteile in.4","Stadtteile in.5"]].rename(columns={"Stadtteile in.4": "District", "Stadtteile in.5": "Rent_per_sqm"}), ignore_index=True)
district_rent_price["Rent_per_sqm"] = district_rent_price["Rent_per_sqm"].str.replace(",", ".").str.replace(" €", "").astype(float)

pd.set_option('display.max_rows', None)

#Price split for Sachsenhausen Nord and Sud not available 
district_rent_price.loc[district_rent_price.District == "Sachsenhausen", "District"] = "Sachsenhausen-Süd"
district_rent_price.loc[45] = ["Sachsenhausen-Nord", float(district_rent_price.loc[district_rent_price.District == "Sachsenhausen-Süd"].Rent_per_sqm)]
#Altstadt price same as Innenstadt
district_rent_price.loc[46] = ["Altstadt", float(district_rent_price.loc[district_rent_price.District == "Innenstadt"].Rent_per_sqm)]

district_rent_price.loc[district_rent_price.District == "Kalbach", "District"] = "Kalbach-Riedberg"

# Merge back to original df
df_districts = pd.merge(df_districts, district_rent_price, on="District")

In [133]:
df_districts

,District,Surface_km2,population,female_pop,male_pop,german_pop,foreign_pop,Latitude,Longitude,Coordinates,geometry,Rent_per_sqm
0,Altstadt,506,4218,2065,2153,2669,1549,50.11044,8.68235,"[[8.6748666, 50.1092226], [8.6757349, 50.10751...","POLYGON ((8.67487 50.10922, 8.67573 50.10752, ...",19.46
1,Innenstadt,1491,6599,3112,3487,3539,3060,50.11456,8.68359,"[[8.6685139, 50.1132235], [8.6690588, 50.11253...","POLYGON ((8.66851 50.11322, 8.66906 50.11253, ...",19.46
2,Bahnhofsviertel,542,3552,1321,2231,1706,1846,50.10841,8.66815,"[[8.6605492, 50.1095394], [8.6625518, 50.10855...","POLYGON ((8.66055 50.10954, 8.66255 50.10856, ...",19.06
3,Westend-Süd,2497,19314,9839,9475,14006,5308,50.11524,8.66227,"[[8.6433346, 50.1140438], [8.6434858, 50.10917...","POLYGON ((8.64333 50.11404, 8.64349 50.10918, ...",18.11
4,Westend-Nord,1632,10373,5391,4982,7366,3007,50.12636,8.66792,"[[8.6558043, 50.1266272], [8.6564224, 50.12597...","POLYGON ((8.65580 50.12663, 8.65642 50.12597, ...",17.41
5,Nordend-West,3100,30897,15845,15052,24144,6753,50.12491,8.67795,"[[8.6714178, 50.1360727], [8.6719059, 50.13375...","POLYGON ((8.67142 50.13607, 8.67191 50.13375, ...",17.64
6,Nordend-Ost,1532,23182,12045,11137,18016,5166,50.12492,8.69232,"[[8.6887636, 50.1185902], [8.6891034, 50.11829...","POLYGON ((8.68876 50.11859, 8.68910 50.11830, ...",19.32
7,Ostend,5564,29477,15108,14369,21070,8407,50.11237,8.69997,"[[8.6941197, 50.1103561], [8.694791, 50.107693...","POLYGON ((8.69412 50.11036, 8.69479 50.10769, ...",19.95
8,Bornheim,2786,30917,16316,14601,23511,7406,50.12973,8.71061,"[[8.6982605, 50.1391582], [8.7003042, 50.13853...","POLYGON ((8.69826 50.13916, 8.70030 50.13853, ...",16.28
9,Gutleutviertel,1792,6964,3047,3917,4000,2964,50.10171,8.66054,"[[8.6215196, 50.0932461], [8.6219131, 50.08876...","POLYGON ((8.62152 50.09325, 8.62191 50.08877, ...",19.64


## Create a map of Frankfurt with neighborhoods superimposed on top


In [14]:
address = 'Frankfurt am Main, DE'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Frankfurt are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Frankfurt are 50.1106444, 8.6820917.


In [18]:
import folium
from urllib.request import urlopen
import json

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# file name - file is located in the working directory
#with urlopen('https://offenedaten.frankfurt.de/dataset/85b38876-729c-4a78-910c-a52d5c6df8d2/resource/84dff094-ab75-431f-8c64-39606672f1da/download/ffmstadtteilewahlen.geojson') as response:
#    districts = json.load(response) 
#communities_geo = districts # geojson file

# create a plain Frankfur map
communities_map = folium.Map(location=[latitude, longitude], zoom_start=11)

#generate choropleth map 
folium.Choropleth(
    geo_data=df_districts,
    data=df_districts[['District', "Rent_per_sqm"]],
    columns=['District', 'Rent_per_sqm'],
    key_on='feature.properties.District',
    fill_color='YlOrRd', 
    fill_opacity=1, 
    line_opacity=1,
    legend_name='Rent per sqm',
    smooth_factor=0).add_to(communities_map)

# display map
#communities_map
# add markers to map
for lat, lng, district in zip(df_districts['Latitude'], df_districts['Longitude'], df_districts['District']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(communities_map)  


style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    df_districts,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['District', 'Rent_per_sqm'],
        aliases=['Neighborhood', 'Rent per sqm'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)

communities_map.add_child(NIL)
communities_map.keep_in_front(NIL)
folium.LayerControl().add_to(communities_map)
communities_map

## Explore Neighborhoods in Frankfurt

### Define Foursquare Credentials and Version

In [134]:
CLIENT_ID = 'LNU4ZAZSYSFL2PM0WJ2WIR2IHSMVSOYA434C2UOVKHCIWXYI' # your Foursquare ID
CLIENT_SECRET = 'T44NCMGFDJCJVPDVFJLIWTYFGGKYA22HKRHWI0PU2JBP10JS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LNU4ZAZSYSFL2PM0WJ2WIR2IHSMVSOYA434C2UOVKHCIWXYI
CLIENT_SECRET:T44NCMGFDJCJVPDVFJLIWTYFGGKYA22HKRHWI0PU2JBP10JS


#### Function to  get the venues by neighborhoods

In [135]:
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [136]:
frankfurt_venues = getNearbyVenues(names=df_districts['District'],
                                   latitudes=df_districts['Latitude'],
                                   longitudes=df_districts['Longitude']
                                  )

Altstadt
Innenstadt
Bahnhofsviertel
Westend-Süd
Westend-Nord
Nordend-West
Nordend-Ost
Ostend
Bornheim
Gutleutviertel
Gallus
Bockenheim
Sachsenhausen-Nord
Sachsenhausen-Süd
Flughafen
Oberrad
Niederrad
Schwanheim
Griesheim
Rödelheim
Hausen
Praunheim
Heddernheim
Niederursel
Ginnheim
Dornbusch
Eschersheim
Eckenheim
Preungesheim
Bonames
Berkersheim
Riederwald
Seckbach
Fechenheim
Höchst
Nied
Sindlingen
Zeilsheim
Unterliederbach
Sossenheim
Nieder-Erlenbach
Kalbach-Riedberg
Harheim
Nieder-Eschbach
Bergen-Enkheim
Frankfurter Berg


In [137]:
print(frankfurt_venues.shape)
frankfurt_venues.head()

(1484, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altstadt,50.11044,8.68235,SCHIRN Kunsthalle,50.110291,8.683542,Art Museum
1,Altstadt,50.11044,8.68235,Römerberg,50.110489,8.682131,Plaza
2,Altstadt,50.11044,8.68235,Dom Aussichtsplattform,50.110609,8.684908,Scenic Lookout
3,Altstadt,50.11044,8.68235,Carhartt,50.111929,8.681853,Boutique
4,Altstadt,50.11044,8.68235,Hoppenworth & Ploch,50.110891,8.683701,Café


In [138]:
frankfurt_venues.groupby('District').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Altstadt,100,100,100,100,100,100
Bahnhofsviertel,100,100,100,100,100,100
Bergen-Enkheim,5,5,5,5,5,5
Berkersheim,3,3,3,3,3,3
Bockenheim,92,92,92,92,92,92
Bonames,10,10,10,10,10,10
Bornheim,65,65,65,65,65,65
Dornbusch,32,32,32,32,32,32
Eckenheim,11,11,11,11,11,11


In [139]:
print('There are {} uniques categories.'.format(len(frankfurt_venues['Venue Category'].unique())))

There are 220 uniques categories.


## Analyze Each Neighborhood

In [140]:
# one hot encoding
frankfurt_onehot = pd.get_dummies(frankfurt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
frankfurt_onehot['District'] = frankfurt_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [frankfurt_onehot.columns[-1]] + list(frankfurt_onehot.columns[:-1])
frankfurt_onehot = frankfurt_onehot[fixed_columns]

frankfurt_onehot.head()

,District,Accessories Store,African Restaurant,Airport Lounge,Airport Service,American Restaurant,Apple Wine Pub,Argentinian Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Castle,Chinese Restaurant,Chocolate Shop,Cigkofte Place,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Residence Hall,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Currywurst Joint,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Duty-free Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Friterie,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hawaiian Restaurant,Health Food Store,History Museum,Hockey Field,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Lawyer,Library,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Malga,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Nightclub,Opera House,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Plaza,Pool,Portuguese Restaurant,Post Office,Pub,Racetrack,Radio Station,Ramen Restaurant,Restaurant,River,Road,Roof Deck,Sake Bar,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Transportation Service,Trattoria/Osteria,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Yoga Studio,Zoo,Zoo Exhibit
0,Altstadt,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Altstadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Altstadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [141]:
frankfurt_grouped = frankfurt_onehot.groupby('District').mean().reset_index()
frankfurt_grouped

,District,Accessories Store,African Restaurant,Airport Lounge,Airport Service,American Restaurant,Apple Wine Pub,Argentinian Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Castle,Chinese Restaurant,Chocolate Shop,Cigkofte Place,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Residence Hall,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Currywurst Joint,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Duty-free Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Friterie,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hawaiian Restaurant,Health Food Store,History Museum,Hockey Field,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Lawyer,Library,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Malga,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Nightclub,Opera House,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Plaza,Pool,Portuguese Restaurant,Post Office,Pub,Racetrack,Radio Station,Ramen Restaurant,Restaurant,River,Road,Roof Deck,Sake Bar,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Transportation Service,Trattoria/Osteria,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfront,Whisky Bar,Wine Bar,Yoga Studio,Zoo,Zoo Exhibit
0,Altstadt,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.02,0.000000,0.000000,0.020000,0.000000,0.030000,0.01,0.000000,0.01,0.000000,0.010000,0.000000,0.010000,0.000000,0.03,0.000000,0.00,0.000000,0.0000,0.000,0.00,0.010000,0.000000,0.00,0.00000,0.000000,0.100000,0.000000,0.000000,0.000000,0.01,0.00,0.000000,0.01,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00000,0.010000,0.00,0.000000,0.01,0.00,0.020000,0.010000,0.010000,0.000000,0.000000,0.010,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.0100,0.000000,0.000000,0.000000,0.0,0.010,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.040000,0.000000,0.010000,0.010000,0.010000,0.000000,0.020000,0.000000,0.00,0.000,0.010000,0.0000,0.00000,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.040000,0.000000,0.00,0.000000,0.000

#### Let's print each neighborhood along with the top 5 most common venues

In [143]:
num_top_venues = 5

for hood in frankfurt_grouped['District']:
    print("----"+hood+"----")
    temp = frankfurt_grouped[frankfurt_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Altstadt----
                venue  freq
0                Café  0.10
1               Plaza  0.04
2         Coffee Shop  0.04
3  Italian Restaurant  0.04
4   German Restaurant  0.04


----Bahnhofsviertel----
                 venue  freq
0                Hotel  0.12
1                  Bar  0.07
2           Restaurant  0.05
3    Indian Restaurant  0.04
4  Japanese Restaurant  0.03


----Bergen-Enkheim----
               venue  freq
0   Business Service   0.2
1     Ice Cream Shop   0.2
2         Water Park   0.2
3            Taverna   0.2
4  German Restaurant   0.2


----Berkersheim----
                        venue  freq
0          Light Rail Station  0.33
1  Construction & Landscaping  0.33
2           German Restaurant  0.33
3            Pedestrian Plaza  0.00
4                      Museum  0.00


----Bockenheim----
                venue  freq
0                Café  0.11
1  Italian Restaurant  0.09
2    Asian Restaurant  0.05
3               Hotel  0.03
4         Supermarket  0.03



#### Let's put that into a _pandas_ dataframe

In [156]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [157]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
districts_venues_sorted = pd.DataFrame(columns=columns)
districts_venues_sorted['District'] = frankfurt_grouped['District']

for ind in np.arange(frankfurt_grouped.shape[0]):
    districts_venues_sorted.iloc[ind, 1:] = return_most_common_venues(frankfurt_grouped.iloc[ind, :], num_top_venues)

districts_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Altstadt,Café,German Restaurant,Plaza,Coffee Shop,Italian Restaurant,Bar,Boutique,Wine Bar,Art Museum,Bakery
1,Bahnhofsviertel,Hotel,Bar,Restaurant,Indian Restaurant,Japanese Restaurant,Vietnamese Restaurant,Café,Coffee Shop,Seafood Restaurant,French Restaurant
2,Bergen-Enkheim,Ice Cream Shop,Water Park,Taverna,Business Service,German Restaurant,Event Service,Food Truck,Food Court,Food & Drink Shop,Food
3,Berkersheim,Construction & Landscaping,German Restaurant,Light Rail Station,Zoo Exhibit,Event Service,Food Truck,Food Court,Food & Drink Shop,Food,Fast Food Restaurant
4,Bockenheim,Café,Italian Restaurant,Asian Restaurant,Wine Bar,Supermarket,Hotel,Pizza Place,Platform,Spanish Restaurant,Ice Cream Shop


## 4. Cluster Neighborhoods

In [158]:
# set number of clusters
kclusters = 10

frankfurt_grouped_clustering = frankfurt_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(frankfurt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 9, 3, 1, 1, 1, 1, 1, 1])

In [159]:
# add clustering labels
districts_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

frankfurt_merged = df_districts

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
frankfurt_merged = frankfurt_merged.join(districts_venues_sorted.set_index('District'), on='District')

frankfurt_merged.head() # check the last columns!

,District,Surface_km2,population,female_pop,male_pop,german_pop,foreign_pop,Latitude,Longitude,Coordinates,geometry,Rent_per_sqm,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Altstadt,506,4218,2065,2153,2669,1549,50.11044,8.68235,"[[8.6748666, 50.1092226], [8.6757349, 50.10751...","POLYGON ((8.67487 50.10922, 8.67573 50.10752, ...",19.46,1,Café,German Restaurant,Plaza,Coffee Shop,Italian Restaurant,Bar,Boutique,Wine Bar,Art Museum,Bakery
1,Innenstadt,1491,6599,3112,3487,3539,3060,50.11456,8.68359,"[[8.6685139, 50.1132235], [8.6690588, 50.11253...","POLYGON ((8.66851 50.11322, 8.66906 50.11253, ...",19.46,1,Café,Italian Restaurant,Coffee Shop,Plaza,Boutique,Hotel,Thai Restaurant,Park,Nightclub,Market
2,Bahnhofsviertel,542,3552,1321,2231,1706,1846,50.10841,8.66815,"[[8.6605492, 50.1095394], [8.6625518, 50.10855...","POLYGON ((8.66055 50.10954, 8.66255 50.10856, ...",19.06,1,Hotel,Bar,Restaurant,Indian Restaurant,Japanese Restaurant,Vietnamese Restaurant,Café,Coffee Shop,Seafood Restaurant,French Restaurant
3,Westend-Süd,2497,19314,9839,9475,14006,5308,50.11524,8.66227,"[[8.6433346, 50.1140438], [8.6434858, 50.10917...","POLYGON ((8.64333 50.11404, 8.64349 50.10918, ...",18.11,1,Italian Restaurant,Steakhouse,Café,Hotel,Park,Chinese Restaurant,Plaza,Indian Restaurant,Fountain,Restaurant
4,Westend-Nord,1632,10373,5391,4982,7366,3007,50.12636,8.66792,"[[8.6558043, 50.1266272], [8.6564224, 50.12597...","POLYGON ((8.65580 50.12663, 8.65642 50.12597, ...",17.41,1,Italian Restaurant,Café,Supermarket,German Restaurant,Bus Stop,Bakery,Park,Coffee Shop,Thai Restaurant,Metro Station


In [160]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(frankfurt_merged['Latitude'], frankfurt_merged['Longitude'], frankfurt_merged['District'], frankfurt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [155]:
(frankfurt_merged.columns.str()

SyntaxError: unexpected EOF while parsing (<ipython-input-155-fa7e2c2c6a6e>, line 1)

In [115]:
frankfurt_merged.loc[frankfurt_merged['Cluster Labels'] == 0, frankfurt_merged.columns.str.contains("Venue|District")].head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Altstadt,Café,Coffee Shop,Plaza,Art Museum,Wine Bar,Restaurant,German Restaurant,Burger Joint,Boutique,Scenic Lookout
1,Innenstadt,Café,Italian Restaurant,Restaurant,Bar,Plaza,Coffee Shop,Thai Restaurant,Hotel,Gym / Fitness Center,Indie Movie Theater
2,Bahnhofsviertel,Hotel,Bar,Indian Restaurant,Restaurant,Café,Vietnamese Restaurant,Asian Restaurant,Seafood Restaurant,Chinese Restaurant,Burger Joint
3,Westend-Süd,Italian Restaurant,Steakhouse,Bakery,Indian Restaurant,Plaza,Organic Grocery,Restaurant,Chinese Restaurant,Café,French Restaurant
4,Westend-Nord,Café,Coffee Shop,Italian Restaurant,Gastropub,German Restaurant,Sushi Restaurant,Garden,Donut Shop,Thai Restaurant,Trail


In [116]:
frankfurt_merged.loc[frankfurt_merged['Cluster Labels'] == 1, frankfurt_merged.columns.str.contains("Venue|District")].head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Sachsenhausen-Süd,Lake,Tram Station,German Restaurant,Food,Czech Restaurant,Cosmetics Shop,Farmers Market,Falafel Restaurant,Exhibit,Event Space
15,Oberrad,Tram Station,German Restaurant,Italian Restaurant,Market,Liquor Store,Bakery,Plaza,Ice Cream Shop,Currywurst Joint,Falafel Restaurant
33,Fechenheim,Campground,German Restaurant,Pizza Place,Bakery,Tram Station,Farmers Market,Falafel Restaurant,Exhibit,Event Space,Ethiopian Restaurant
35,Nied,Tram Station,Pizza Place,Italian Restaurant,Supermarket,Electronics Store,Bar,Light Rail Station,German Restaurant,Dessert Shop,Dim Sum Restaurant


In [117]:
frankfurt_merged.loc[frankfurt_merged['Cluster Labels'] == 2, frankfurt_merged.columns.str.contains("Venue|District")].head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Bergen-Enkheim,Bus Stop,Zoo Exhibit,Construction & Landscaping,Food,Farmers Market,Falafel Restaurant,Exhibit,Event Space,Ethiopian Restaurant,Electronics Store


In [118]:
frankfurt_merged.loc[frankfurt_merged['Cluster Labels'] == 3, frankfurt_merged.columns.str.contains("Venue|District")].head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Niederursel,German Restaurant,Pizza Place,Supermarket,Zoo Exhibit,Farmers Market,Falafel Restaurant,Exhibit,Event Space,Ethiopian Restaurant,Electronics Store
32,Seckbach,Supermarket,Plaza,Pub,Dog Run,German Restaurant,Cosmetics Shop,Doner Restaurant,Falafel Restaurant,Exhibit,Event Space


In [119]:
frankfurt_merged.loc[frankfurt_merged['Cluster Labels'] == 4, frankfurt_merged.columns.str.contains("Venue|District")].head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Griesheim,Supermarket,Bakery,Doner Restaurant,Yoga Studio,Pie Shop,Electronics Store,River,Light Rail Station,Zoo Exhibit,Falafel Restaurant
21,Praunheim,Supermarket,German Restaurant,Racetrack,Restaurant,Italian Restaurant,Liquor Store,Dog Run,Falafel Restaurant,Exhibit,Event Space
22,Heddernheim,German Restaurant,Insurance Office,Falafel Restaurant,Supermarket,Bakery,Metro Station,Pizza Place,Dance Studio,Deli / Bodega,Farmers Market
24,Ginnheim,Supermarket,Plaza,Trattoria/Osteria,Metro Station,Music Store,Bakery,Doner Restaurant,Asian Restaurant,Italian Restaurant,German Restaurant
28,Preungesheim,Metro Station,Supermarket,Automotive Shop,Gym / Fitness Center,Climbing Gym,Liquor Store,Zoo Exhibit,Donut Shop,Farmers Market,Falafel Restaurant


In [103]:
frankfurt_merged.loc[frankfurt_merged['Cluster Labels'] == 5, frankfurt_merged.columns.str.contains("Venue|District")].head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,Nieder-Erlenbach,Hotel,Pizza Place,Apple Wine Pub,Scenic Lookout,Doner Restaurant,Farmers Market,Falafel Restaurant,Exhibit,Event Space,Ethiopian Restaurant


In [163]:
frankfurt_merged.columns[frankfurt_merged.columns.str.contains("Venue")]

Index(['1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue'],
      dtype='object')

In [168]:
import folium
from urllib.request import urlopen
import json

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# file name - file is located in the working directory
#with urlopen('https://offenedaten.frankfurt.de/dataset/85b38876-729c-4a78-910c-a52d5c6df8d2/resource/84dff094-ab75-431f-8c64-39606672f1da/download/ffmstadtteilewahlen.geojson') as response:
#    districts = json.load(response) 
#communities_geo = districts # geojson file

# create a plain Frankfur map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

#generate choropleth map 
folium.Choropleth(
    geo_data=frankfurt_merged,
    data=frankfurt_merged[['District', "Rent_per_sqm"]],
    columns=['District', 'Rent_per_sqm'],
    key_on='feature.properties.District',
    fill_color='YlOrRd', 
    fill_opacity=1, 
    line_opacity=1,
    legend_name='Rent per sqm',
    smooth_factor=0).add_to(map_clusters)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(frankfurt_merged['Latitude'], frankfurt_merged['Longitude'], frankfurt_merged['District'], frankfurt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters) 
    
    
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    frankfurt_merged,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['District', 'Rent_per_sqm', 'Cluster Labels'] + frankfurt_merged.columns[frankfurt_merged.columns.str.contains("Venue")].to_list(),
        aliases=['Neighborhood', 'Rent per sqm', 'Cluster'] + frankfurt_merged.columns[frankfurt_merged.columns.str.contains("Venue")].to_list(),
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)

map_clusters.add_child(NIL)
map_clusters.keep_in_front(NIL)
folium.LayerControl().add_to(map_clusters)
map_clusters

In [130]:
frankfurt_venues

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altstadt,50.11044,8.68235,SCHIRN Kunsthalle,50.110291,8.683542,Art Museum
1,Altstadt,50.11044,8.68235,Römerberg,50.110489,8.682131,Plaza
2,Altstadt,50.11044,8.68235,Dom Aussichtsplattform,50.110609,8.684908,Scenic Lookout
3,Altstadt,50.11044,8.68235,Carhartt,50.111929,8.681853,Boutique
4,Altstadt,50.11044,8.68235,Hoppenworth & Ploch,50.110891,8.683701,Café
5,Altstadt,50.11044,8.68235,Weinterasse Rollanderhof,50.112473,8.682164,Wine Bar
6,Altstadt,50.11044,8.68235,Bitter & Zart Chocolaterie,50.111444,8.683904,Chocolate Shop
7,Altstadt,50.11044,8.68235,Main,50.108390,8.682631,River
8,Altstadt,50.11044,8.68235,Kleinmarkthalle,50.112778,8.682958,Market
9,Altstadt,50.11044,8.68235,Wackers Kaffee,50.112064,8.679467,Coffee Shop
